In [3]:
from src.utils import download_images

In [4]:
import pandas as pd
df = pd.read_csv('train.csv')

In [5]:
df_10 = df.sample(10)
df_10

,image_link,group_id,entity_name,entity_value
7106,https://m.media-amazon.com/images/I/8139VF3tG9...,318770,wattage,240.0 watt
82935,https://m.media-amazon.com/images/I/818UfF6cw2...,558374,item_weight,100 gram
199353,https://m.media-amazon.com/images/I/51ujoCaA-z...,368364,depth,7.9 centimetre
3649,https://m.media-amazon.com/images/I/61uTo4TOXM...,179080,wattage,250.0 watt
216264,https://m.media-amazon.com/images/I/51p6z3hU+9...,347320,depth,16.0 inch
185265,https://m.media-amazon.com/images/I/517QGUmQpm...,119607,depth,30.0 millimetre
154197,https://m.media-amazon.com/images/I/51kjS9-Lfc...,752266,depth,14.0 centimetre
198490,https://m.media-amazon.com/images/I/51Mh3RMYrc...,300630,depth,22.2 centimetre
137914,https://m.media-amazon.com/images/I/61EDpvl6Py...,244283,height,14.0 inch
253962,https://m.media-amazon.com/images/I/51mRAHY6y1...,966823,height,14.0 centimetre


In [6]:
images_links = df_10['image_link'].to_list()

In [7]:
download_images(images_links, "./sample_images", True)

Exception in thread Thread-5 (_handle_workers):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\manas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
  0%|          | 0/10 [00:00<?, ?it/s]    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\multiprocessing\pool.py", line 522, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python

In [ ]:
import cv2
import pytesseract
import numpy as np
import matplotlib.pyplot as plt

# Function to display multiple images in a grid
def display_images(images, title):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))  # 2 rows, 5 columns
    fig.suptitle(title, fontsize=16)
    
    for i, img in enumerate(images):
        ax = axes[i // 5, i % 5]
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_title(f'Image {i+1}')
    
    plt.tight_layout()
    plt.show()

# Initialize an array to store the images
images = []

# Define the number of images (10 in this case)
num_images = 10

# Path where the images are stored
image_paths = [f'image_{i+1}.png' for i in range(num_images)]  # Assuming image_1.png, image_2.png, ...

# 1. Load and display original images
original_images = [cv2.imread(path) for path in image_paths]
display_images(original_images, "Original Images")

# 2. Convert to grayscale and display
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in original_images]
display_images(gray_images, "Grayscale Images")

# 3. Apply Otsu's Binarization and display
binary_images = [cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] for gray in gray_images]
display_images(binary_images, "Binarized Images (Otsu's)")

# 4. Noise removal using Median Blur and display
denoised_images = [cv2.medianBlur(binary, 3) for binary in binary_images]
display_images(denoised_images, "Denoised Images (Median Blur)")

# 5. Dilation and display
kernel = np.ones((1, 1), np.uint8)
dilated_images = [cv2.dilate(denoised, kernel, iterations=1) for denoised in denoised_images]
display_images(dilated_images, "Dilated Images")

# 6. Erosion and display
eroded_images = [cv2.erode(dilated, kernel, iterations=1) for dilated in dilated_images]
display_images(eroded_images, "Eroded Images")

# 7. Skew Correction (rotation) and display
def correct_skew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

skew_corrected_images = [correct_skew(eroded) for eroded in eroded_images]
display_images(skew_corrected_images, "Skew Corrected Images")

# 8. Resize images (increase size for better OCR performance)
resized_images = [cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC) for image in skew_corrected_images]
display_images(resized_images, "Resized Images")

# 9. Invert colors for white text on black background (optional, based on image content)
inverted_images = [cv2.bitwise_not(image) for image in resized_images]
display_images(inverted_images, "Inverted Images (Optional)")

# 10. OCR Processing on each preprocessed image
ocr_results = []
for img in inverted_images:
    text = pytesseract.image_to_string(img)
    ocr_results.append(text)

# Output OCR results for each image
for idx, text in enumerate(ocr_results):
    print(f"Text from image {idx+1}:\n{text}\n")
